In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import pylab
import matplotlib as mpl
import numpy as np
import lnm
from graph import Graph
import ipywidgets as widgets
from IPython.display import display, HTML
mpl.rcParams['savefig.dpi'] = 150
OPTIONS = {'FSM (HEAD)': 'results_fsm_head.txt',
           'KCFA (6.2.1)': 'results_kcfa_6_2_1.txt',
           'KCFA (HEAD)': 'results_kcfa_head.txt',
           'Morse Code (HEAD)': 'results_morse-code_head.txt',
           'Snake (HEAD)': 'results_snake_head.txt',
           'Snake (6.2.1)': 'results_snake_6_2_1.txt',
           'Tetris (6.2.1)': 'results_tetris_6_2_1.txt',
           'Tetris (HEAD)': 'results_tetris_head.txt',
           'Tetris (HEAD+UNROLL)': 'results_tetris_head_unroll.txt',
           'Sieve (6.2.1)': 'results_sieve_6_2_1.txt',
           'Sieve (HEAD)': 'results_sieve_head.txt',
           'Suffix Tree (6.2.1)': 'results_suffixtree_6_2_1.txt',
           'Suffix Tree (HEAD)': 'results_suffixtree_head.txt',
           'Synth (HEAD)':'results_synth_head.txt',
           'Synth (6.2.1)': 'results_synth_6_2_1.txt'}
s = widgets.Dropdown(options=OPTIONS)
all_data_files = ['results_fsm_head.txt',
                  'results_morse-code_head.txt',
                  'results_snake_head.txt',
                  'results_tetris_head_unroll.txt',
                  'results_sieve_head.txt',
                  'results_kcfa_head.txt',
                  'results_suffixtree_head.txt',
                  'results_synth_head.txt']
def strip_name(name):
    return name.rpartition("_")[0].partition("_")[2]

In [ ]:
display(s)

In [ ]:
benchmark = strip_name(s.value)
HTML("<h1>Benchmark: '" + benchmark + "'</h1>")

# Select Benchmark:

In [ ]:
graph = lnm.read_data(s.value)
variations, data = graph.ungraph()
hidden_untyped = data[0,2]
pycket_untyped = data[0,1]
racket_untyped = data[0,0]
hidden = data[:,2]
pycket = data[:,1]
racket = data[:,0]

In [ ]:
# Runtime of Pycket as a function of Racket
ordered_data = np.copy(data)
ordered_data.sort(axis=0)
ohidden = ordered_data[:,2]
opycket = ordered_data[:,1]
oracket = ordered_data[:,0]
pylab.plot(oracket, ohidden, 'r', label='hidden')
pylab.plot(oracket, opycket, 'g', label='pycket')
pylab.plot(oracket, oracket, 'b', label='racket')
pylab.title('runtime as a function of Racket runtime')
pylab.xlabel('Racket runtime (ms)')
pylab.ylabel('system runtime (ms)')
pylab.legend(loc='best')

In [ ]:
from stats import *
    
rs_, ps_, hs_, weights_ = zip(*[make_slowdown_data(fname) for fname in all_data_files])
rs = reduce(np.append, rs_)
ps = reduce(np.append, ps_)
hs = reduce(np.append, hs_)
weights = reduce(np.append, weights_)
print weights.shape, rs.shape, ps.shape, hs.shape

# All Data

## All Benchmarks for Hidden

In [ ]:
slowdown_cdf(*[(arr, None, strip_name(name)) for arr, name in zip(hs_, all_data_files)],
            name="name",
            many_weights=weights_)

## All Benchmarks for Pycket

In [ ]:
slowdown_cdf(*[(arr, None, strip_name(name)) for arr, name in zip(ps_, all_data_files)],
            name="name",
            many_weights=weights_)

## All Benchmarks for Racket

In [ ]:
slowdown_cdf(*[(arr, None, strip_name(name)) for arr, name in zip(rs_, all_data_files)],
            name="name",
            many_weights=weights_)

In [ ]:
print hs.shape, weights.shape
slowdown_cdf((rs, 'b', 'racket'),
             (ps, 'g', 'pycket'),
             (hs, 'r', 'hidden'),
             weights=weights,
             name="all_data")

# Slowdown Summary Stats

In [ ]:
DataFrame({'racket':racket, 'pycket':pycket, 'hidden':hidden}).describe()

# Deliverability Stats

## L = 0

In [ ]:
slowdown_graph = lnm.fromkeyvals(variations, racket_slowdown, pycket_slowdown, hidden_slowdown)
racket_slowdown0, pycket_slowdown0, hidden_slowdown0, tbl = compute_lnm_deliverable(slowdown_graph, 0)
tbl

## L = 1

In [ ]:
racket_slowdown1, pycket_slowdown1, hidden_slowdown1, tbl = compute_lnm_deliverable(slowdown_graph, 1)
tbl

## L = 2

In [ ]:
racket_slowdown2, pycket_slowdown2, hidden_slowdown2, tbl = compute_lnm_deliverable(slowdown_graph, 2)
tbl

# Slowdown distribution

## L = 0

In [ ]:
slowdown_cdf((racket_slowdown0, 'b', 'racket'),
             (pycket_slowdown0, 'g', 'pycket'),
             (hidden_slowdown0, 'r', 'hidden'),
             name="%s_l=0" % benchmark)

## L = 1

In [ ]:
slowdown_cdf((racket_slowdown1, 'b', 'racket'),
             (pycket_slowdown1, 'g', 'pycket'),
             (hidden_slowdown1, 'r', 'hidden'),
             name="%s_l=1" % benchmark)

## L = 2

In [ ]:
slowdown_cdf((racket_slowdown2, 'b', 'racket'),
             (pycket_slowdown2, 'g', 'pycket'),
             (hidden_slowdown2, 'r', 'hidden'),
             name="%s_l=2" % benchmark)